In [1]:
from tqdm import tqdm
import pathlib
import json
import numpy as np
import pandas as pd

In [2]:
dataset_directory = pathlib.Path('../input/otto-recommender-system')

In [3]:
type_dict = {
    'clicks': 0,
    'carts': 1,
    'orders': 2
}

def create_ground_truth(events):
    
    """
    Create ground-truth labels from given list of event dictionaries
    
    Parameters
    ----------
    events: list of shape (n_events)
        List of event dictionaries
 
    Returns
    -------
    events: list of shape (n_events)
        List of event dictionaries with labels
    """
    
    previous_labels = {'clicks': None, 'carts': set(), 'orders': set()}

    for event in reversed(events):
        
        event['labels'] = {}

        for label in ['clicks', 'carts', 'orders']:
            if previous_labels[label]:
                if label != 'clicks':
                    event['labels'][type_dict[label]] = previous_labels[label].copy()
                else:
                    event['labels'][type_dict[label]] = previous_labels[label]

        if event['type'] == 'clicks':
            previous_labels['clicks'] = event['aid']
        if event['type'] == 'carts':
            previous_labels['carts'].add(event['aid'])
        elif event['type'] == 'orders':
            previous_labels['orders'].add(event['aid'])

    return events


def create_dataframe(json_file_path, chunk_size):
    
    """
    Create pandas.DataFrame from given json_file_path
    
    Parameters
    ----------
    json_file_path: path-like str
        Path of the json file
        
    chunk_size: int
        Size of chunks while reading the json file
        
    Returns
    -------
    df: pandas.DataFrame of shape (n_samples, 4)
    """
    
    chunks = pd.read_json(json_file_path, lines=True, chunksize=chunk_size)
    df = pd.DataFrame()
    
    for chunk_idx, chunk in enumerate(chunks):
        
        print(f'Reading Chunk {chunk_idx} ({chunk_size * chunk_idx}-{(chunk_size * (chunk_idx + 1))})')
        
        event_dict = {
            'session': [],
            'aid': [],
            'ts': [],
            'type': []
        }
        
        for session, events in tqdm(zip(chunk['session'].tolist(), chunk['events'].tolist()), total=len(chunk['session'].tolist())):
            
            for event in events:
                
                event_dict['session'].append(session)
                event_dict['aid'].append(event['aid'])
                event_dict['ts'].append(event['ts'])
                event_dict['type'].append(type_dict[event['type']])

        chunk_session = pd.DataFrame(event_dict)
        chunk_session['session'] = chunk_session['session'].astype(np.uint32)
        chunk_session['aid'] = chunk_session['aid'].astype(np.uint32)
        chunk_session['ts'] = pd.to_datetime(chunk_session['ts'], unit='ms')
        chunk_session['type'] = chunk_session['type'].astype(np.uint8)
        df = pd.concat([df, chunk_session])
        
    df.reset_index(drop=True, inplace=True)
        
    return df


In [4]:
df_train = create_dataframe(json_file_path=str(dataset_directory / 'train.jsonl'), chunk_size=100000)
df_train

Reading Chunk 0 (0-100000)


100%|██████████| 100000/100000 [00:03<00:00, 26652.03it/s]


Reading Chunk 1 (100000-200000)


100%|██████████| 100000/100000 [00:03<00:00, 27253.85it/s]


Reading Chunk 2 (200000-300000)


...

Reading Chunk 127 (12700000-12800000)


100%|██████████| 100000/100000 [00:00<00:00, 106897.42it/s]


Reading Chunk 128 (12800000-12900000)


100%|██████████| 99779/99779 [00:00<00:00, 116810.58it/s]


,session,aid,ts,type
0,0,1517085,2022-07-31 22:00:00.025,0
1,0,1563459,2022-07-31 22:01:44.511,0
2,0,1309446,2022-08-01 15:23:59.426,0
3,0,16246,2022-08-01 15:28:39.997,0
4,0,1781822,2022-08-01 15:31:11.344,0
...,...,...,...,...
216716091,12899776,1737908,2022-08-28 21:59:47.073,0
216716092,12899777,384045,2022-08-28 21:59:36.974,0
216716093,12899777,384045,2022-08-28 21:59:46.800,0
216716094,12899778,561560,2022-08-28 21:59:43.611,0


In [5]:
df_test = create_dataframe(json_file_path=str(dataset_directory / 'test.jsonl'), chunk_size=100000)
df_test

Reading Chunk 0 (0-100000)


100%|██████████| 100000/100000 [00:00<00:00, 256490.16it/s]


Reading Chunk 1 (100000-200000)


100%|██████████| 100000/100000 [00:00<00:00, 242153.61it/s]


Reading Chunk 2 (200000-300000)


...

Reading Chunk 15 (1500000-1600000)


100%|██████████| 100000/100000 [00:00<00:00, 209623.59it/s]


Reading Chunk 16 (1600000-1700000)


100%|██████████| 71803/71803 [00:00<00:00, 223093.91it/s]


,session,aid,ts,type
0,12899779,59625,2022-08-28 22:00:00.278,0
1,12899780,1142000,2022-08-28 22:00:00.378,0
2,12899780,582732,2022-08-28 22:00:58.352,0
3,12899780,973453,2022-08-28 22:01:49.199,0
4,12899780,736515,2022-08-28 22:02:16.868,0
...,...,...,...,...
6928118,14571577,1141710,2022-09-04 21:59:34.770,0
6928119,14571578,519105,2022-09-04 21:59:35.009,0
6928120,14571579,739876,2022-09-04 21:59:35.605,0
6928121,14571580,202353,2022-09-04 21:59:41.067,0


In [6]:
print(f'Training Shape: {df_train.shape} - Memory Usage: {df_train.memory_usage().sum() / 1024 ** 2:.2f} MB')
print(f'Test Shape: {df_test.shape} - Memory Usage: {df_test.memory_usage().sum() / 1024 ** 2:.2f} MB')

Training Shape: (216716096, 4) - Memory Usage: 3513.50 MB
Test Shape: (6928123, 4) - Memory Usage: 112.32 MB


In [7]:
%%time

df_train.to_pickle('train.pkl')
df_test.to_pickle('test.pkl')

CPU times: user 1.49 s, sys: 9.15 s, total: 10.6 s
Wall time: 10.7 s
